In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Thu May 27 11:58:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os
!rm -r transformers

rm: cannot remove 'transformers': No such file or directory


In [4]:
import torch
print(torch.__version__)

1.8.1+cu101


In [5]:
!cp drive/MyDrive/zindi_nlp/MalawiNews/*.csv .

In [ ]:
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

In [6]:
!pip install git+https://github.com/huggingface/transformers.git
#transformers==4.5transformers==4.6.0.dev0

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-qzjw4vnh
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-qzjw4vnh
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 901kB 17.3MB/s 
     |████████████████████████████████| 3.3MB 51.6MB/s 
  Created wheel for transformers: filename=transformers-4.7.0.dev0-cp37-none-any.whl size=2316570 sha256=6cd318509264487a20887f44ab1cf1984d3d7213a15f2fdf1ddd0d002af71343
  Stored in directory: /tmp/pip-ephem-wheel-cache-9fbn8ykl/wheels/33/eb/3b/4bf5dd835e865e472d4fc0754f35ac0edb08fe852e8f21655f
Successfully built transformers


In [7]:
!pip install datasets sentencepiece 

     |████████████████████████████████| 235kB 14.6MB/s 
     |████████████████████████████████| 1.2MB 22.5MB/s 
     |████████████████████████████████| 112kB 55.0MB/s 
     |████████████████████████████████| 245kB 58.4MB/s 


In [8]:
!pip install emoji

     |████████████████████████████████| 133kB 14.4MB/s 


In [9]:
! git clone https://github.com/huggingface/transformers.git
# && cd transformers && git checkout 4906a29 
#  bae0c79

Cloning into 'transformers'...
remote: Enumerating objects: 73558, done.
remote: Counting objects: 100% (311/311), done.
remote: Compressing objects: 100% (200/200), done.
remote: Total 73558 (delta 150), reused 183 (delta 91), pack-reused 73247
Receiving objects: 100% (73558/73558), 56.63 MiB | 18.67 MiB/s, done.
Resolving deltas: 100% (52251/52251), done.


In [10]:
import pandas as pd
import numpy as np
import warnings
from tqdm.auto import tqdm
warnings.simplefilter("ignore")

data = pd.read_csv("Train.csv").Text.tolist() + pd.read_csv("Test.csv").Text.tolist()
print(len(data))
print(len(pd.read_csv("Train.csv")), len(pd.read_csv("Test.csv")))

2056
1436 620


In [11]:
texts = []
for x in tqdm(data):
  texts.extend(x.strip().split("\n"))


In [12]:
#%load_ext tensorboard

In [13]:
with open("datas.txt", "w") as f:
  for line in texts:
    f.write(line.strip() + "\n")


In [14]:
#%tensorboard --logdir runs

In [ ]:
!python transformers/examples/pytorch/language-modeling/run_mlm.py --model_name_or_path "setu4993/LaBSE" --max_seq_length 200 --gradient_accumulation 8 --fp16 \
  --train_file "datas.txt" --validation_file "datas.txt"  --do_train --per_device_train_batch_size 4  --save_steps 15000 --do_eval --line_by_line --seed 42 \
  --learning_rate 1e-5 --mlm_probability 0.2 --num_train_epochs 2000  --output_dir drive/MyDrive/zindi_nlp/MalawiNews/pretrain/LABSE-v100-reproduce/ --preprocessing_num_workers=4\
  #--pad_to_max_length True   # transformers/examples/xla_spawn.py --num_cores 8 

  # 60k bs 4 4->8 max 200

2021-05-27 12:02:48.935402: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
05/27/2021 12:02:51 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
05/27/2021 12:02:51 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=drive/MyDrive/zindi_nlp/MalawiNews/pretrain/LABSE-v100-reproduce/, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=4, per_device_eval_batch_size=8, gradient_accumulation_steps=8, eval_accumulation_steps=None, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=2000.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/May27_12-02-51_f1cb4bc3684c, logging_s

In [ ]:
!python transformers/examples/language-modeling/run_mlm.py -h